# Kaggle Competition: Natural Language Processing with Disaster Tweets


## Description of the Problem and Data

### Problem
(from Kaggle) Twitter has become an important communication channel in times of emergency.
The ubiquitousness of smartphones enables people to announce an emergency they’re observing in real-time. Because of this, more agencies are interested in programatically monitoring Twitter (i.e. disaster relief organizations and news agencies).

### Data
Each sample in the train and test set has the following information: The text of a tweet, a keyword from that tweet (watch for blanks!), and the location the tweet was sent from (also could be missing).

We are predicting whether a given tweet is about a real disaster or not.  If so, predict a '1', otherwise '0'.

Columns: 'id', 'text', 'location', 'keyword', 'target' (in train.csv only, this denotes whether a tweet is about a real disaster or not).

## Exploratory Data Analysis

## Analysis (Model Building and Training)

## Result

Submissions are evaluated using F1 between the predicted and expected answers.

## Discussion and Conclusion